# 5. Ground and road fusion

In [ ]:
# Standard library and path imports
import logging
import os

# Third-party library imports
from upcp.pipeline import Pipeline
from upcp.labels import Labels
import upcp.fusion as fusion
import upcp.utils.ahn_utils as ahn_utils
import upcp.utils.bgt_utils as bgt_utils
import upcp.utils.las_utils as las_utils
import upcp.utils.log_utils as log_utils
import upcp.utils.csv_utils as csv_utils
import upcp.scrapers.ams_bgt_scraper as ams_bgt_scraper
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()

# Local or project-specific imports
import settings as st

if st.my_run == "azure":
    import config_azure as cf
elif st.my_run == "local":
    import config as cf

# INFO messages will be printed to console.
log_utils.reset_logger()
log_utils.add_console_logger(level=logging.DEBUG)

## Select area to label point clouds for

In [ ]:
# Import areas
df_areas = gpd.read_file(cf.output_pilot_area)
polygon = df_areas.to_crs(crs=st.CRS).unary_union

## Collect point clouds in area

In [ ]:
# If running via Azure, mount to Azure point cloud storage here
ahn_in_folder = cf.ahn_folder
bgt_in_folder = cf.bgt_folder
in_folder_point_clouds = cf.in_folder_point_clouds
out_folder_point_clouds = cf.out_folder_point_clouds

for path in [bgt_in_folder, out_folder_point_clouds]:
    if not os.path.isdir(path):
        os.makedirs(path)

In [ ]:
# Collect all tiles in polygon area
all_pc_filenames = []
for path, subdirs, files in os.walk(in_folder_point_clouds):
    for name in files:
            if '.laz' in name:
                all_pc_filenames.append(os.path.join(path, name))
all_pc_filenames = np.array(all_pc_filenames)
all_pc_tilecodes = np.array([las_utils.get_tilecode_from_filename(filename) for filename in all_pc_filenames])
all_pc_bboxes = np.array([las_utils.get_bbox_from_tile_code(tilecode) for tilecode in all_pc_tilecodes])

all_pc_polygons = [las_utils.get_polygon_from_tile_code(tilecode) for tilecode in all_pc_tilecodes]
gdf_pc_polygons = gpd.GeoDataFrame(geometry=all_pc_polygons)

gdf = gdf_pc_polygons.intersection(polygon)
pc_idxs_in_area_polygon = gdf[~gdf.is_empty].index.to_list()

# filenames, tilecodes and bounding boxes of point clouds in selected area
pc_filenames = all_pc_filenames[pc_idxs_in_area_polygon]
pc_tilecodes = all_pc_tilecodes[pc_idxs_in_area_polygon]
pc_bboxes = all_pc_bboxes[pc_idxs_in_area_polygon]

sns.set()
ax = gdf_pc_polygons.iloc[pc_idxs_in_area_polygon].boundary.plot()
ax = df_areas.boundary.plot(ax=ax)
plt.title('Point clouds in selected area')
plt.show()

## Ground and building fuser using pre-processed BGT and AHN data

In [ ]:
# Write all BGT files for tiles in selected area if they do not already exist. 
# If cell is interupted due to too many api requests, restart kernel and run all cells again to infer missing BGT files.
for pc_tilecode, pc_bbox in tqdm(zip(pc_tilecodes, pc_bboxes), total=len(pc_tilecodes)):

    # Output file for the BGT fuser.
    bgt_data_file = bgt_in_folder + '{}.csv'.format(pc_tilecode)
    if not os.path.isfile(bgt_data_file):
        csv_headers = ['bgt_name', 'polygon', 'x_min', 'y_max', 'x_max', 'y_min']
            
        # Road and parking spots layers in BGT
        bgt_layers = ['BGT_WGL_rijbaan_lokale_weg', 'BGT_WGL_rijbaan_regionale_weg',
                    'BGT_WGL_rijbaan_autoweg', 'BGT_WGL_rijbaan_autosnelweg',
                    'BGT_WGL_parkeervlak', 'BGT_WGL_ov-baan', 'BGT_WGL_fietspad']

        # Scrape data from the Amsterdam WFS and parse the json.
        bgt_road_polygons_csv = []
        for layer in bgt_layers:
            # Scrape data from the Amsterdam WFS, this will return a json response.
            json_content = ams_bgt_scraper.scrape_amsterdam_bgt(layer, bbox=pc_bbox)
            
            # Parse the downloaded json response.
            parsed_content = ams_bgt_scraper.parse_polygons(json_content)
            bgt_road_polygons_csv += parsed_content

        # Write the csv
        csv_utils.write_csv(bgt_data_file, bgt_road_polygons_csv, csv_headers)

In [ ]:
# Create the reader for .npz data.
npz_reader = ahn_utils.NPZReader(ahn_in_folder)

# Label point clouds
for tilecode, filename in tqdm(zip(pc_tilecodes, pc_filenames), total=len(pc_tilecodes)):

    # get file directories
    bgt_road_file = bgt_in_folder + tilecode + '.csv'
    pc_in_file = filename 
    pc_out_file = out_folder_point_clouds + 'road_ground_labeled_' + tilecode + '.laz'
    if not os.path.isfile(pc_out_file):

        # Create reader for BGT road part polygons.
        bgt_road_reader = bgt_utils.BGTPolyReader(bgt_file=bgt_road_file)

        # Create fusers
        params = {'bottom': 0., 'buffer': 0.02}
        npz_ground_fuser = fusion.AHNFuser(Labels.GROUND, ahn_reader=npz_reader,
                                    target='ground', epsilon=0.2, params=params)
        road_part_fuser = fusion.BGTRoadFuser(Labels.ROAD, bgt_reader=bgt_road_reader)

        # Pipeline to label ground and road 
        process_sequence = (npz_ground_fuser, road_part_fuser)
        pipeline = Pipeline(processors=process_sequence, caching=False)

        # Process the file.
        pipeline.process_file(pc_in_file, out_file=pc_out_file)